# Imports

In [39]:
# Importing required libraries

from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, RepeatedKFold, RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, classification_report, make_scorer, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
reg_signals = pd.read_csv('data/indicators.csv', parse_dates=True, index_col='Date')
poly_features = pd.read_csv('data/indicators_w_polyterms.csv', parse_dates=True, index_col='Date')
labels = pd.read_csv('data/Y_Matrix.csv', parse_dates=True, index_col='Date')

# Model Preparation

In [5]:
# Defining the period rolling window over which we train new models and how far back the training data goes

window = 1
lookback = 3

# Creating a function to generate the training and testing data for the models, from 2000 onwards we create for every single year a train test split
# by using last 3 years as training data and the next year as testing data
# Eg. the model that will trade between 2000 and 2001 will be trained on data from 1997 to 1999 and tuned on 1999 to 2000 data (for the purposes of hyperparameter tuning)

def generate_data(reg_signals, labels, window, lookback):
    '''
    Function to generate training and testing data for the models
    
    Parameters:
    reg_signals: DataFrame containing the  signals
    labels: DataFrame containing the labels
    window: int, number of years to use for the testing data
    lookback: int, number of years to use for the training data
    
    '''
    X = reg_signals
    y = labels
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    for i in range(2000, 2025):
        date = datetime.strptime(str(i) + '-01-01', '%Y-%m-%d')
        X_train.append(X.loc[str(date - pd.DateOffset(years=lookback)):str(date - pd.DateOffset(years=window))])
        X_test.append(X.loc[str(date - pd.DateOffset(years=window)):str(date)])
        y_train.append(y.loc[str(date - pd.DateOffset(years=lookback)):str(date - pd.DateOffset(years=window))])
        y_test.append(y.loc[str(date - pd.DateOffset(years=window)):str(date)])
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = generate_data(reg_signals, labels, window, lookback)
X_poly_train, X_poly_test, y_poly_train, y_poly_test = generate_data(poly_features, labels, window, lookback)

# NB: y_train and y_poly_train are the same for all models, as the y_backtest and y_poly_backtest are the same for all models, so we just keep one set
del y_poly_train, y_poly_test

In [6]:
# Tuning function for one single pipeline for one single period. We use the same training and testing data for all models to ensure that the results are comparable.
# We use the predefined X_train, X_test split instead of cross-validation to avoid look-ahead biases.

def tune_model(X_train, X_test, y_train, y_test, params, pipeline, hyperparameter_tuner='grid', n_iter=None, verbose=1, sample_weights=None, probability=True):
    '''
    Function to tune a model using a predefined split
    
    Parameters:
    X_train: DataFrame, training data
    X_test: DataFrame, testing data over which we perform the hyperparameter tuning
    y_train: DataFrame, training labels
    y_test: DataFrame, testing labels
    params: dict, hyperparameters to tune
    pipeline: Pipeline, model to tune
    hyperparameter_tuner: str, 'grid' or 'random', default='grid'
    n_iter: int, number of iterations for random search, default=None
    verbose: int, default=1
    sample_weights: array-like, default=None
    probability: bool, whether the model has a predict_proba method, default=True
    
    '''
    # we first merge the training and testing data
    X = pd.concat([X_train, X_test])
    y = pd.concat([y_train, y_test])
    # our crossvalidation cv uses predefined split to ensure that the training and testing data are actually maintained when tuning hyperparameters:
    cv = PredefinedSplit(test_fold=[-1]*len(X_train) + [0]*len(X_test))
     
    if probability:
        scoring = {'auc': make_scorer(roc_auc_score, needs_proba=True, multi_class="ovr"), 'f1_macro': 'f1_macro', 'precision': 'precision_macro', 'recall': 'recall_macro', 'balanced_accuracy': 'balanced_accuracy'}
    else: 
        scoring = {'f1_macro': 'f1_macro', 'precision': 'precision_macro', 'recall': 'recall_macro', 'balanced_accuracy': 'balanced_accuracy'}
    principal_metric = 'balanced_accuracy'

    if hyperparameter_tuner == 'grid':
        search = GridSearchCV(pipeline, params, cv=cv, scoring=scoring, refit=principal_metric, n_jobs=-1, verbose=verbose, error_score='raise')
    elif hyperparameter_tuner == 'random':
        search = RandomizedSearchCV(pipeline, params, cv=cv, scoring=scoring, refit=principal_metric, n_iter=n_iter, n_jobs=-1, verbose=verbose)
    else:
        raise ValueError(f"Unknown hyperparameter tuner: {hyperparameter_tuner}\n Choose from 'grid', 'random'")
    if sample_weights is not None:
        kwargs = {pipeline.steps[-1][0] + '__sample_weight': sample_weights}
        result = search.fit(X, y, **kwargs)
    else:
        result = search.fit(X, y)
    best_model = result.best_estimator_

    return result, best_model

In [7]:
def model_validation_performance(result, probability=True):
    print('')
    print(f"Cross-validation balanced accuracy scores: {result.best_score_}\n")
    print(f"Cross-validation F1 Macro scores: {result.cv_results_['mean_test_f1_macro'][result.best_index_]}\n")
    if probability:
        print(f"Cross-validation ROC AUC scores: {result.cv_results_['mean_test_auc'][result.best_index_]}\n")
    print(f"Cross-validation precision scores: {result.cv_results_['mean_test_precision'][result.best_index_]}\n")
    print(f"Cross-validation recall scores: {result.cv_results_['mean_test_recall'][result.best_index_]}\n")
    print(f"Parameters for the best model: \n{result.best_params_}")

In [34]:
def rolling_model_tuning(X_train, X_test, y_train, y_test, params, pipeline, hyperparameter_tuner='grid', n_iter=None, verbose=1, sample_weights=None, probability=True):
    '''
    Function to tune a model using a rolling window
    
    Parameters:
    X_train: list, training data
    X_test: list, testing data over which we perform the hyperparameter tuning
    y_train: list, training labels
    y_test: list, testing labels
    params: dict, hyperparameters to tune
    pipeline: Pipeline, model to tune
    hyperparameter_tuner: str, 'grid' or 'random', default='grid'
    n_iter: int, number of iterations for random search, default=None
    verbose: int, default=1
    sample_weights: array-like, default=None
    probability: bool, whether the model has a predict_proba method, default=True
    
    '''
    results = dict()
    best_models = dict()
    for target in y_train[0].columns:
        print(f"Training model for target {target}\n")
        for i in range(len(X_train)):
            result, best_model = tune_model(X_train[i], X_test[i], y_train[i][target], y_test[i][target], params, pipeline, hyperparameter_tuner=hyperparameter_tuner, n_iter=n_iter, verbose=verbose, sample_weights=sample_weights, probability=probability)
            results[target] = result
            best_models[target] = best_model
            # we print the results for each model
            print(f"Results for model {i+2000}:")
            if verbose:
                model_validation_performance(result, probability=probability)
            else:
                print(f"Best score: {result.best_score_}")
        print('\n\n\n')
    return results, best_models

# Model Training

### Logistic Regression

In [35]:
# Define the pipeline
log_reg_pipeline = Pipeline(
    [('log_reg', LogisticRegression(max_iter=1000))]
)

# Define the parameters to tune
log_reg_params = [{
    'log_reg__C': [0.01, 0.1, 1, 10, 100],
    'log_reg__solver': ['liblinear'],
    'log_reg__penalty': ['l1', 'l2'],
    'log_reg__class_weight': ['balanced', None]
},
                  {
    'log_reg__C': [0.01, 0.1, 1, 10, 100],
    'log_reg__solver': ['saga'],
    'log_reg__penalty': ['elasticnet'],
    'log_reg__l1_ratio': [0.1, 0.5, 0.9],
    'log_reg__class_weight': ['balanced', None]
                  }]


In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
log_reg_results, log_reg_best_models = rolling_model_tuning(X_train, X_test, y_train, y_test, log_reg_params, log_reg_pipeline, hyperparameter_tuner='grid', verbose=0, probability=True)

Training model for target long 3 days

Results for model 2000:
Best score: 0.6472222222222221
Results for model 2001:
Best score: 0.6481481481481481
Results for model 2002:
Best score: 0.5743194192377495
Results for model 2003:
Best score: 0.6077712609970675
Results for model 2004:
Best score: 0.5634056299030918
Results for model 2005:
Best score: 0.5788785934901401
Results for model 2006:
Best score: 0.6085714285714285
Results for model 2007:
Best score: 0.5418939393939394
Results for model 2008:
Best score: 0.5944720056697378
Results for model 2009:
Best score: 0.5685393258426966
Results for model 2010:
Best score: 0.6228146853146853
Results for model 2011:
Best score: 0.6553813257305774
Results for model 2012:
Best score: 0.6174418604651163
Results for model 2013:
Best score: 0.6175
Results for model 2014:
Best score: 0.5718077474892396
Results for model 2015:
Best score: 0.6296919587327139
Results for model 2016:
Best score: 0.6851983844143502
Results for model 2017:
Best score: 0.

In [41]:
# Save as a pickle file in the models folder

with open('models/log_reg_results.pkl', 'wb') as f:
    pickle.dump(log_reg_results, f)


### Random Forest

In [42]:
# Define the pipeline
rf_pipeline = Pipeline(
    [('rf', RandomForestClassifier())]
)

# Define the parameters to tune
rf_params = {
    'rf__n_estimators': [100, 200, 300, 400, 500],
    'rf__max_depth': [10, 20, 30, 40, 50],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__bootstrap': [True, False],
    'rf__class_weight': ['balanced', None]
}


In [43]:
rf_results, rf_best_models = rolling_model_tuning(X_train, X_test, y_train, y_test, rf_params, rf_pipeline, hyperparameter_tuner='random', n_iter=100, verbose=0, probability=True)

Training model for target long 3 days

Results for model 2000:
Best score: 0.6194444444444445
Results for model 2001:
Best score: 0.5793650793650793
Results for model 2002:
Best score: 0.619600725952813
Results for model 2003:
Best score: 0.5720918866080157
Results for model 2004:
Best score: 0.5335948315643747
Results for model 2005:
Best score: 0.5837491090520314
Results for model 2006:
Best score: 0.5784415584415584
Results for model 2007:
Best score: 0.5663636363636364
Results for model 2008:
Best score: 0.609355067328136
Results for model 2009:
Best score: 0.6107865168539326
Results for model 2010:
Best score: 0.5321241258741258
Results for model 2011:
Best score: 0.5838679021145166
Results for model 2012:
Best score: 0.6072674418604651
Results for model 2013:
Best score: 0.565
Results for model 2014:
Best score: 0.5586800573888092
Results for model 2015:
Best score: 0.6371551913367967
Results for model 2016:
Best score: 0.675694939415538
Results for model 2017:
Best score: 0.5951

In [44]:
# save the results as pickle file in the models folder

with open('models/rf_results.pkl', 'wb') as f:
    pickle.dump(rf_results, f)

### XGBoost

In [15]:
# Define the pipeline
xgb_pipeline = Pipeline(
    [('xgb', XGBClassifier())]
)

# Define the parameters to tune
xgb_params = {
    'xgb__n_estimators': [100, 200, 300, 400, 500],
    'xgb__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'xgb__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'xgb__subsample': [0.5, 0.7, 0.9, 1],
    'xgb__colsample_bytree': [0.5, 0.7, 0.9, 1],
    'xgb__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'xgb__reg_alpha': [0, 0.1, 0.5, 1],
    'xgb__reg_lambda': [0, 0.1, 0.5, 1],
    'xgb__scale_pos_weight': [1, 2, 3, 4, 5],
    'xgb__eval_metric': ['mlogloss', 'merror'],
    'xgb__objective': ['multi:softmax', 'multi:softprob']
}

In [ ]:
xgb_results, xgb_best_models = rolling_model_tuning(X_train, X_test, y_train, y_test, xgb_params, xgb_pipeline, hyperparameter_tuner='random', n_iter=100, verbose=0, probability=True)

In [ ]:
# save the results as pickle file in the models folder

with open('models/xgb_results.pkl', 'wb') as f:
    pickle.dump(xgb_results, f)

### SVM

In [ ]:
# Define the pipeline
svm_pipeline = Pipeline(
    [('svm', SVC(probability=True))]
)

# Define the parameters to tune
svm_params = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'svm__degree': [2, 3, 4, 5],
    'svm__gamma': ['scale', 'auto'],
    'svm__class_weight': ['balanced', None]
}

In [ ]:
svm_results, svm_best_models = rolling_model_tuning(X_train, X_test, y_train, y_test, svm_params, svm_pipeline, hyperparameter_tuner='random', n_iter=100, verbose=0, probability=True)

In [ ]:
# save the results as pickle file in the models folder

with open('models/svm_results.pkl', 'wb') as f:
    pickle.dump(svm_results, f)

# Model Training